In [113]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import numpy as np
import tensorflow as tf



In [114]:
model = tf.keras.models.load_model('model.h5')


In [115]:
from tensorflow.keras.models import load_model
import pickle

model = load_model('model.h5')

with open('onehot_encoder_geo.pickle', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pickle', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pickle', 'rb') as file:
    scaler = pickle.load(file)

In [116]:
import pandas as pd

input_data = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Male',
    'Age':40,
    'Tenure':5,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}


In [117]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Male,40,5,60000,2,1,1,50000


In [118]:
# Example: OneHotEncoder already fitted as label_encoder_geo
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)
geo_encoded_df


c:\Users\saima\Downloads\anaconda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [119]:
input_df = pd.DataFrame([input_data])  # ✅ Convert dict → DataFrame first

# Now safely drop the 'Geography' column and concat with encoded df
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,Male,40,5,60000,2,1,1,50000,1.0,0.0,0.0


In [120]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [121]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.01709861,  0.91324755,  0.10479359, -0.00134472, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [122]:
prediction = model.predict(input_scaled)

print(f"Prediction Probability: {prediction[0][0]:.4f}")

if prediction[0][0] > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Prediction Probability: 0.0376
The customer is not likely to churn.
